In [175]:
#Import libraries
import numpy as np
import pandas as pd


In [176]:
#Loading data
file_path = 'data.npy'
loaded_array = np.load(file_path)

In [177]:
def polar_to_cartesian(r, theta):
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y


In [178]:
x_coord, y_coord = polar_to_cartesian(loaded_array[:,0], loaded_array[:,1])
scores = loaded_array[:,2]

In [179]:
def calc_gradients(x,y):
    x_non_zero = np.where(x == 0, 1, x)
    result = np.divide(y, x_non_zero)
    result = np.where(x == 0, np.nan, result)
    return result


In [180]:
slopes = calc_gradients(x_coord, y_coord)

In [181]:
intersection_y = lambda r,m,theta: r*(np.sin(theta) - m*np.cos(theta))

intersection_values_y = intersection_y(loaded_array[:,0],slopes,loaded_array[:,1])


In [182]:
intersection_x = lambda b,m: -b/m

intersection_values_x = intersection_x(intersection_values_y,slopes)

/var/folders/pn/7xb0xrx119l4_783pgg7gp140000gn/T/ipykernel_3478/1933300804.py:1: RuntimeWarning: invalid value encountered in divide
  intersection_x = lambda b,m: -b/m


In [183]:
#Getting indices of positive and negative oriented lines 

positive_gradient_lines = np.where(slopes[slopes >= 0])[0]
negative_gradient_lines = np.where(slopes[slopes < 0])[0]


In [184]:
#Create a new array that contains the x-intercept difference for each line of positive gradient to each line of negative gradient
x_intercept_diff = np.abs(intersection_values_x[positive_gradient_lines][:, np.newaxis] - intersection_values_x[negative_gradient_lines])
scores_diff = (scores[positive_gradient_lines][:, np.newaxis] + scores[negative_gradient_lines])

In [185]:
closest_pairs_indices = np.unravel_index(np.argsort(x_intercept_diff, axis=None), x_intercept_diff.shape)
closest_pairs_indices = np.column_stack(closest_pairs_indices)[:100]

closest_pairs_data = []

for index_pair in closest_pairs_indices:
    positive_grad = slopes[slopes >= 0][index_pair[0]]
    positive_intersection = intersection_values_x[index_pair[0]]
    negative_grad = slopes[slopes < 0][index_pair[1]]
    negative_intersection = intersection_values_x[index_pair[1]]
    x_intercept_difference = x_intercept_diff[index_pair[0], index_pair[1]]
    sum_score = scores_diff[index_pair[0], index_pair[1]]

    closest_pairs_data.append([
        positive_grad, negative_grad,positive_intersection,negative_intersection,
        x_intercept_difference, sum_score
    ])

column_names = ['Positive Gradient', 'Negative Gradient','Positive Intersection','Negative Intersection',
                'x-intercept Difference', 'Sum Score']

df = pd.DataFrame(closest_pairs_data, columns=column_names)

df

,Positive Gradient,Negative Gradient,Positive Intersection,Negative Intersection,x-intercept Difference,Sum Score
0,0.017455,-1.539865,0.000003,0.000003,0.0,136.0
1,1.072369,-2.747478,-0.000000,-0.000000,0.0,60.0
2,1.072369,-3.077684,-0.000000,-0.000000,0.0,60.0
3,1.072369,-4.010781,-0.000000,-0.000000,0.0,60.0
4,1.072369,-4.704632,-0.000000,-0.000000,0.0,60.0
...,...,...,...,...,...,...
95,1.072369,-2.144506,-0.000000,-0.000000,0.0,57.0
96,1.072369,-2.144506,-0.000000,-0.000000,0.0,57.0
97,1.072369,-2.050304,-0.000000,-0.000000,0.0,57.0
98,1.072369,-3.732051,-0.000000,0.000000,0.0,56.0
